In [278]:
import _expds_tokens

In [279]:
syear='2019'
pyear='2018'
season='2019_games.html'
team_url_csv="nhl_urls.csv"
csv_out='nhl_games_'
season_start=pyear+'-01-01'

In [280]:
from bs4 import BeautifulSoup

def get_current_season_links (url,team,url_ref):
    #url='https://www.baseball-reference.com/teams/ATL/'
    #url_ref='2018-schedule-scores.shtml'
    page = requests.get(url+url_ref) #downloads html contents of the website and stores them
    #page.status_code tells if download was successful. starting with 2 is good, with 4 or 5 means wrong
    #can print content with page.content for html
    soup = BeautifulSoup(page.text, 'html.parser') #parse HTML content for information in <p> tag

    soup_filter=soup.find("table", attrs={'class':'sortable stats_table'}) #find all instances of a certain tag, in this case "div"
    table_body = soup_filter.find('tbody')
    
    l = []
    #Loop over table rows, collect all table data entries, store row text, add to list
    for tr in table_body.find_all('tr',class_='')[0:]:
        tds = tr.find_all('td')
        row = [tr.text for tr in tds]
        l.append(row) 
        
    dfout=pd.DataFrame(l) 
    
    dfout['Team']=team
    
    return dfout  

In [281]:
ds=pd.read_csv(team_url_csv)
team_url_list = ds.iloc[:,0].tolist()
team_url_list = list(set(team_url_list))
team_abrv=[]
for i in team_url_list:
    team_abrv.append(i[-4:-1])
if syear == '2017':
    team_abrv.remove('VEG')

In [282]:
games_df = pd.DataFrame()
for u,w in zip(team_url_list,team_abrv):
    #print(u)
    #print(w)
    dfatl=get_current_season_links(u,w,season) #
    games_df=games_df.append(dfatl) #add new dataframe to empty one

if syear == '2019':
    games_df.columns = ['date_raw', 'home_away', 'opponent','home_score','away_score','result','1','wins','losses','ot_losses','streak','attendance','2','3','home_team_abrv']
else: 
    games_df.columns = ['date_raw', 'time', 'home_away', 'opponent','home_score','away_score','result','1','wins','losses','ot_losses','streak','attendance','2','3','home_team_abrv']
games_df.drop(columns=['1','2','3'], inplace = True)

games_df['date_raw'].replace('', np.nan, inplace=True)
games_df['streak'].replace('', 'N 0', inplace=True)
games_df.dropna(subset=['date_raw'], inplace=True)

In [283]:
games_df.head()

,date_raw,home_away,opponent,home_score,away_score,result,wins,losses,ot_losses,streak,attendance,home_team_abrv
0,2018-10-03,,Calgary Flames,5,2,W,1,0,0,W 1,"18,870",VAN
1,2018-10-06,@,Calgary Flames,4,7,L,1,1,0,L 1,"18,688",VAN
2,2018-10-09,@,Carolina Hurricanes,3,5,L,1,2,0,L 2,"11,932",VAN
3,2018-10-11,@,Tampa Bay Lightning,4,1,W,2,2,0,W 1,"19,092",VAN
4,2018-10-13,@,Florida Panthers,3,2,W,3,2,0,W 2,"11,953",VAN


In [284]:
#Standardize date
split_date = games_df['date_raw'].str.split('-',n=3,expand = True)
        
dates = []
day_of_week = []
for i in range(0,len(split_date[1])):
    day = dt.date(int(split_date[0].iloc[i][-4:]), int(split_date[1].iloc[i]), int(split_date[2].iloc[i]))
    time = day.strftime("%Y-%m-%d")
    day_of_week.append(day.strftime('%A'))
    dates.append(time)
games_df['game_date'] = dates
games_df['day_of_week'] = day_of_week

In [285]:
#weekend
week_ends = []

for i in range(0,len(games_df['day_of_week'])):
    if games_df['day_of_week'].iloc[i] == 'Friday':
        week_ends.append(1)
    elif games_df['day_of_week'].iloc[i] == 'Saturday':
        week_ends.append(1)
    elif games_df['day_of_week'].iloc[i] == 'Sunday':
        week_ends.append(1)
    else: week_ends.append(0)
        
games_df['weekend_flag'] = week_ends

In [286]:
first_game=games_df.groupby('home_team_abrv',as_index=False).first()
first_game.head()
first_game=first_game[['home_team_abrv','game_date']]

first_game_flag = []
for index, row in first_game.iterrows():
    first_game_flag.append(1)
first_game['first_game_flag']=first_game_flag

games_df = pd.merge(games_df,first_game,on=['home_team_abrv','game_date'], how='left')
games_df['first_game_flag'].replace(np.nan, 0, inplace=True)

In [287]:
#home or away
home_or_away = []
for index, row in games_df.iterrows():
    if row['home_away']=='@': home_or_away.append(0)
    else: home_or_away.append(1)
    
games_df['home_flag'] = home_or_away

In [288]:
#Streaks
streaks = []
i = 0
#len(games_df.streak_raw)
for i in range(0,len(games_df.streak)):
    streaks.append(int(str(games_df.streak.iloc[i])[-1]))
    if games_df.streak.iloc[i].find("L") == 0:
        streaks[i] = streaks[i] * -1
    else:
        streaks[i] = streaks[i]
games_df['streak'] = streaks

In [289]:
#Join in group_id
games_df_merge = pd.merge(games_df, ds,  how='left', left_on=['home_team_abrv'], right_on = ['team_abbreviation'])

In [290]:
games_df_merge.head()

,date_raw,home_away,opponent,home_score,away_score,result,wins,losses,ot_losses,streak,...,group_id,city,zip_code_us,group_config_id,group_type_id,short_name,alert_name,billing_category_id,home_demand_tier,away_demand_tier
0,2018-10-03,,Calgary Flames,5,2,W,1,0,0,1,...,309.0,"Vancouver, BC",NaN,170.0,9.0,VANC,Canucks Alert,6.0,2,2
1,2018-10-06,@,Calgary Flames,4,7,L,1,1,0,-1,...,309.0,"Vancouver, BC",NaN,170.0,9.0,VANC,Canucks Alert,6.0,2,2
2,2018-10-09,@,Carolina Hurricanes,3,5,L,1,2,0,-2,...,309.0,"Vancouver, BC",NaN,170.0,9.0,VANC,Canucks Alert,6.0,2,2
3,2018-10-11,@,Tampa Bay Lightning,4,1,W,2,2,0,1,...,309.0,"Vancouver, BC",NaN,170.0,9.0,VANC,Canucks Alert,6.0,2,2
4,2018-10-13,@,Florida Panthers,3,2,W,3,2,0,2,...,309.0,"Vancouver, BC",NaN,170.0,9.0,VANC,Canucks Alert,6.0,2,2


In [291]:
from dotenv import load_dotenv
import os
from pathlib import Path 

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

postgres_user = os.environ['PSQL_USERNAME']
postgres_pw = os.environ['PSQL_PASSWORD']
postgres_url = os.environ['PSQL_URL']

redshift_user = os.environ['REDSHIFT_USERNAME']
redshift_pw = os.environ['REDSHIFT_PASSWORD']
redshift_url = os.environ['REDSHIFT_URL']

t_consumer_key = os.environ['TWT_CONSUMER_KEY']
t_consumer_secret = os.environ['TWT_CONSUMER_SECRET']
t_access_token = os.environ['TWT_ACCESS_TOKEN_KEY']
t_access_token_secret = os.environ['TWT_ACCESS_TOKEN_SECRET']

In [292]:
#this block makes the connection to postgres db (tableau/follower)
conn_string = "host='"+postgres_url+"' dbname='experience' user='"+postgres_user+"' password='"+postgres_pw+"'"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

In [293]:
#this block makes the connection to redshift (redluma)
red_engine = create_engine('postgresql://'+redshift_user+':'+redshift_pw+'@'+redshift_url)
conn_red = red_engine.raw_connection()
cur = conn_red.cursor()

In [294]:
statement="""
select g.* 
    ,gr.*
    ,TO_CHAR(g.game_start_time - INTERVAL '5 hours','YYYY-MM-DD') AS game_date
    ,gr.id as group_id
    ,g.id as game_id
    ,v.name as venue
    ,v.max_attendance 
  from game g
  join "group" gr
  on g.home_team_id=gr.id and gr.group_type_id=9
  left join venue v
  on gr.id=v.group_id
where g.game_start_time>'{}';
""".format(season_start)
games_exp = pd.read_sql(statement, conn)
games_exp.to_csv('nhl_games_test_right.csv')

In [295]:
#left join games_df to games_exp on date, game_date group_ip, home_team_id
games_df_merge_full = pd.merge(games_df_merge, games_exp,  how='left', on=['group_id','game_date'])

In [296]:
statement="""
select 
 CAST(game_id AS INTEGER) as game_id
,group_name
,final_event_name
,away_team_fix
,away_demand_tier as away_demand_tier_tm
,home_demand_tier as home_demand_tier_tm
,sum(seats_sold) as seats_sold
,sum(seats_scanned) as scanned_seats
,sum(section_seats) as assumed_capacity

from temp_attn_rates
where segment='NHL'
group by 1,2,3,4,5,6;
"""
sales_scan = pd.read_sql(statement, conn_red)

In [297]:
games_df_merge_full_sales = pd.merge(games_df_merge_full, sales_scan,  how='left', on=['game_id'])
games_df_merge_full_sales.dropna(subset=['group_name'], inplace=True)
games_df_merge_full_sales.to_csv('nhl_games_test2.csv')

In [298]:
pd.set_option('display.max_rows', 100)
games_df_merge_full_sales.dtypes

date_raw                         object
home_away                        object
opponent                         object
home_score                       object
away_score                       object
result                           object
wins                             object
losses                           object
ot_losses                        object
streak                            int64
attendance                       object
home_team_abrv                   object
game_date                        object
day_of_week                      object
weekend_flag                      int64
first_game_flag                 float64
home_flag                         int64
url                              object
team_abbreviation                object
team_name_expapp                 object
team_name_web                    object
group_id                        float64
city_x                           object
zip_code_us                     float64
group_config_id_x               float64


In [299]:
game_feature=games_df_merge_full_sales[[\
'result',\
'home_score',\
'away_score',\
'wins',\
'losses',\
'streak',\
'home_team_abrv',\
'game_date',\
'weekend_flag',\
'first_game_flag',\
'day_of_week',\
'group_id',\
'game_id',\
'away_team',\
'name',\
'max_attendance',\
'away_demand_tier',\
'home_demand_tier',\
'seats_sold',\
'scanned_seats',\
'assumed_capacity']]
game_feature=game_feature.astype({'wins': 'int32'})
game_feature=game_feature.astype({'losses': 'int32'})
game_feature=game_feature.astype({'away_demand_tier': 'int32'})
game_feature=game_feature.astype({'home_demand_tier': 'int32'})

In [300]:
#current win/loss ratio
ratios = []
for index, row in game_feature.iterrows():
    ratios.append(int(row['wins'])/(int(row['wins'])+int(row['losses']))) 
    
game_feature['wl_ratio'] = ratios

In [301]:
#%neg streak days
days = []
neg_days = []
gt_two_pos_days = []
for index, row in game_feature.iterrows():
    if row['streak']<0 : neg_days.append(1)
    else : neg_days.append(0)
    if row['streak']>2 : gt_two_pos_days.append(1)
    else : gt_two_pos_days.append(0)
    days.append(1)
    
game_feature['neg_streak_days'] = neg_days
game_feature['pos_streak_gt_2_days'] = gt_two_pos_days
game_feature['days'] = days

game_feature['c_neg_streak_days']=game_feature.groupby(['home_team_abrv'])['neg_streak_days'].cumsum()
game_feature['c_pos_streak_gt_2_days']=game_feature.groupby(['home_team_abrv'])['pos_streak_gt_2_days'].cumsum()
game_feature['c_days']=game_feature.groupby(['home_team_abrv'])['days'].cumsum()

game_feature['pct_neg_streak']=game_feature['c_neg_streak_days']/game_feature['c_days']
game_feature['pct_pos_streak_gt_2_days']=game_feature['c_pos_streak_gt_2_days']/game_feature['c_days']

game_feature['c10_neg_streak_days']=game_feature.groupby('home_team_abrv')['neg_streak_days'].rolling(10).sum().reset_index(0,drop=True)
game_feature['c10_pos_streak_gt_2_days']=game_feature.groupby('home_team_abrv')['pos_streak_gt_2_days'].rolling(10).sum().reset_index(0,drop=True)

roll_neg_days = []
roll_gt_two_pos_days = []
for index, row in game_feature.iterrows():
    if row['c10_neg_streak_days'] >=0: roll_neg_days.append(row['c10_neg_streak_days']/row['c_days']) 
    else : roll_neg_days.append(row['pct_neg_streak'])
    if row['c10_pos_streak_gt_2_days'] >=0 : roll_gt_two_pos_days.append(row['c10_pos_streak_gt_2_days']/row['c_days'])
    else : roll_gt_two_pos_days.append(row['pct_pos_streak_gt_2_days']) 

game_feature['pct_c10_neg_streak_days'] = roll_neg_days
game_feature['pct_c10_pos_streak_gt_2_days'] = roll_gt_two_pos_days

In [302]:
#day_of_week_flag(s)
dummy=pd.get_dummies(game_feature['day_of_week'])
game_feature=pd.concat([game_feature,dummy], axis=1)

In [303]:
#home_team_tier x away_team_tier
tiers = []
for index, row in game_feature.iterrows():
    tiers.append('h_'+str(row['home_demand_tier'])+'_a_'+str(row['away_demand_tier']))
game_feature['tier_combo'] = tiers

dummy=pd.get_dummies(game_feature['tier_combo'])
game_feature=pd.concat([game_feature,dummy], axis=1)

In [304]:
#fix GSW arena move
#if syear=='2019': 
#    game_feature.loc[game_feature['home_team_abrv'] == 'GSW', 'assumed_capacity'] = 19500
#    game_feature.loc[game_feature['home_team_abrv'] == 'BOS', 'assumed_capacity'] = 18624
#Is this block useful for capacity changes in the NHL?

In [305]:
#avg sell-through
#avg scan-through
sell_through = []
scan_through = []
scan_rate = []
for index, row in game_feature.iterrows():
    sell_through.append(row['seats_sold']/row['assumed_capacity'])
    scan_through.append(row['scanned_seats']/row['assumed_capacity'])
    scan_rate.append(row['scanned_seats']/row['seats_sold'])
    
game_feature['sell_through'] = sell_through
game_feature['scan_through'] = scan_through
game_feature['scan_rate'] = scan_rate

game_feature['c_sell_through']=game_feature.groupby(['home_team_abrv'])['sell_through'].cumsum()
game_feature['c_scan_through']=game_feature.groupby(['home_team_abrv'])['scan_through'].cumsum()
game_feature['c_scan_rate']=game_feature.groupby(['home_team_abrv'])['scan_rate'].cumsum()
#need to know avg for the row above though as you don't know that game's rates to add to avg

sell_through_avg = []
scan_through_avg = []
scan_rate_avg = []
for index, row in game_feature.iterrows():
    sell_through_avg.append(row['c_sell_through']/row['c_days'])
    scan_through_avg.append(row['c_scan_through']/row['c_days'])
    scan_rate_avg.append(row['c_scan_rate']/row['c_days'])

game_feature['c_sell_through_avg'] = sell_through_avg
game_feature['c_scan_through_avg'] = scan_through_avg
game_feature['c_scan_rate_avg'] = scan_rate_avg    

#shift down
sell_through_shift = []
scan_through_shift = []
scan_rate_shift = []

sell_through_prev = None
scan_through_prev = None
scan_rate_prev = None
grp = ''
for index, row in game_feature.iterrows():
    if row['home_team_abrv'] == grp : 
        sell_through_shift.append(sell_through_prev) 
        scan_through_shift.append(scan_through_prev) 
        scan_rate_shift.append(scan_rate_prev)
    else : 
        sell_through_shift.append(None)
        scan_through_shift.append(None)
        scan_rate_shift.append(None)
    
    sell_through_prev = row['c_sell_through_avg']
    scan_through_prev = row['c_scan_through_avg']
    scan_rate_prev = row['c_scan_rate_avg']
    grp = row['home_team_abrv']

game_feature['c_sell_through_avg_shift'] = sell_through_shift
game_feature['c_scan_through_avg_shift'] = scan_through_shift
game_feature['c_scan_rate_avg_shift'] = scan_rate_shift  

avg_sell_through_fix = game_feature['c_sell_through_avg_shift'].fillna(game_feature.groupby(['home_team_abrv'])['c_sell_through_avg_shift'].transform('mean'))
avg_scan_through_fix = game_feature['c_scan_through_avg_shift'].fillna(game_feature.groupby(['home_team_abrv'])['c_scan_through_avg_shift'].transform('mean'))
avg_scan_rate_fix = game_feature['c_scan_rate_avg_shift'].fillna(game_feature.groupby(['home_team_abrv'])['c_scan_rate_avg_shift'].transform('mean'))

game_feature['c_sell_through_avg_shift_fix']=avg_sell_through_fix
game_feature['c_scan_through_avg_shift_fix']=avg_scan_through_fix
game_feature['c_scan_rate_avg_shift_fix']=avg_scan_rate_fix

In [306]:
#features list
#current win/loss ratio
#%neg streak days
#%positive streak days
#%3+ game streaks
#%neg streak days last 10 games
#%positive streak days last 10 games
#%3+ game streaks last 10 games
#day_of_week
#home_demand_tier 
#away_demand_tier
#weekend
#average scan rate

In [307]:
#game_feature.to_csv('game_feature_nba.csv')

In [308]:
#standard score on sell_through, scan_through, scan_rate across season
#join back 
#create z score for each game
stdev_sell_through=game_feature.groupby(['home_team_abrv'])['sell_through'].agg(np.std, ddof=0).to_frame()
stdev_scan_through=game_feature.groupby(['home_team_abrv'])['scan_through'].agg(np.std, ddof=0).to_frame()
stdev_scan_rate=game_feature.groupby(['home_team_abrv'])['scan_rate'].agg(np.std, ddof=0).to_frame()

avg_sell_through=game_feature.groupby(['home_team_abrv'])['sell_through'].agg(np.average).to_frame()
avg_scan_through=game_feature.groupby(['home_team_abrv'])['scan_through'].agg(np.average).to_frame()
avg_scan_rate=game_feature.groupby(['home_team_abrv'])['scan_rate'].agg(np.average).to_frame()

stdev_sell_through.rename(columns = {'sell_through':'sell_through_stdev'}, inplace = True)
stdev_scan_through.rename(columns = {'scan_through':'scan_through_stdev'}, inplace = True)
stdev_scan_rate.rename(columns = {'scan_rate':'scan_rate_stdev'}, inplace = True)

avg_sell_through.rename(columns = {'sell_through':'sell_through_avg'}, inplace = True)
avg_scan_through.rename(columns = {'scan_through':'scan_through_avg'}, inplace = True)
avg_scan_rate.rename(columns = {'scan_rate':'scan_rate_avg'}, inplace = True)

game_feature = pd.merge(game_feature, stdev_sell_through,  how='left', on=['home_team_abrv'])
game_feature = pd.merge(game_feature, stdev_scan_through,  how='left', on=['home_team_abrv'])
game_feature = pd.merge(game_feature, stdev_scan_rate,  how='left', on=['home_team_abrv'])

game_feature = pd.merge(game_feature, avg_sell_through,  how='left', on=['home_team_abrv'])
game_feature = pd.merge(game_feature, avg_scan_through,  how='left', on=['home_team_abrv'])
game_feature = pd.merge(game_feature, avg_scan_rate,  how='left', on=['home_team_abrv'])


d = {'sell_through_stdev': ['max']\
     ,'scan_through_stdev': ['max']\
     ,'scan_rate_stdev': ['max']\
     ,'sell_through_avg': ['max']\
     ,'scan_through_avg': ['max']\
     ,'scan_rate_avg': ['max']\
    }

rates_meta=game_feature.groupby(['home_team_abrv']).agg(d)

sell_through_z = []
scan_through_z = []
scan_rate_z = []

for index, row in game_feature.iterrows():
    sell_through_z.append((row['sell_through']-row['sell_through_avg'])/row['sell_through_stdev'])
    scan_through_z.append((row['scan_through']-row['scan_through_avg'])/row['scan_through_stdev'])
    scan_rate_z.append((row['scan_rate']-row['scan_rate_avg'])/row['scan_rate_stdev'])

game_feature['sell_through_std'] = sell_through_z
game_feature['scan_through_std'] = scan_through_z
game_feature['scan_rate_std'] = scan_rate_z

In [309]:
game_feature.to_csv('game_feature_nhl.csv')
rates_meta.to_csv('rates_meta_nhl.csv')

In [310]:
#keep just columns for prediction
game_feature_predict=game_feature[[\
'streak',\
'home_team_abrv',\
'game_date',\
'weekend_flag',\
'first_game_flag',\
'group_id',\
'game_id',\
'away_team',\
'away_demand_tier',\
'home_demand_tier',\
'seats_sold',\
'scanned_seats',\
'assumed_capacity',\
'wl_ratio',\
'pct_neg_streak',\
'pct_pos_streak_gt_2_days',\
'pct_c10_neg_streak_days',\
'pct_c10_pos_streak_gt_2_days',\
'Friday',\
'Monday',\
'Saturday',\
'Sunday',\
'Thursday',\
'Tuesday',\
'Wednesday',\
'h_1_a_1',\
'h_1_a_2',\
'h_1_a_3',\
'h_2_a_1',\
'h_2_a_2',\
'h_2_a_3',\
'h_3_a_1',\
'h_3_a_2',\
'h_3_a_3',\
'sell_through',\
'scan_through',\
'scan_rate',\
'c_sell_through_avg_shift_fix',\
'c_scan_through_avg_shift_fix',\
'c_scan_rate_avg_shift_fix',\
'sell_through_avg',\
'scan_through_avg',\
'scan_rate_avg',\
'sell_through_std',\
'scan_through_std',\
'scan_rate_std']]

In [311]:
game_feature_predict.to_csv('game_feature_predict_nhl.csv')